In [1]:
import pandas as pd

# Step 1: Load RNA, protein, and phospho dataframes
mrna_df = pd.read_csv("data_input/mrna.csv", index_col=0)
proteo_df = pd.read_csv("data_input/proteo.csv", index_col=0)
phospho_df = pd.read_csv("data_input/phospho.csv", index_col=0)
metadata_df = pd.read_csv("data_input/metadata.csv", index_col=0)

# Strip whitespace from column names
mrna_df.columns = mrna_df.columns.str.strip()
proteo_df.columns = proteo_df.columns.str.strip()
phospho_df.columns = phospho_df.columns.str.strip()

# Step 2: Clean column names for proteomics and phosphoproteomics data

# For proteomics data (remove "iBAQ " prefix)
proteo_df.columns = [col.replace("iBAQ ", "") for col in proteo_df.columns]

# For phosphoproteomics data (remove "Intensity " prefix and "_phos" suffix)
phospho_df.columns = [col.replace("Intensity ", "").replace("_phos", "") for col in phospho_df.columns]

# Step 3: Get sample name lists
mrna_list = list(mrna_df.columns)
proteo_list = list(proteo_df.columns)
phospho_list = list(phospho_df.columns)

common_values = set(mrna_list) & set(proteo_list) & set(phospho_list)
strings_to_filter = list(common_values)

#strings_to_filter

# Step 5: Filter the dataframes to keep only common samples
mrna_df_filtered = mrna_df.loc[:, mrna_df.columns.isin(strings_to_filter)]
proteo_df_filtered = proteo_df.loc[:, proteo_df.columns.isin(strings_to_filter)]
phospho_df_filtered = phospho_df.loc[:, phospho_df.columns.isin(strings_to_filter)]

# Save filtered dataframes as CSV
mrna_df_filtered.to_csv("data_output/mrna_filtered.csv")
phospho_df_filtered.to_csv("data_output/phospho_filtered.csv")
proteo_df_filtered.to_csv("data_output/proteo_filtered.csv")

#proteo_df_filtered
#mrna_df_filtered

In [2]:
# Transpose and clean function
def format_expression_matrix(df):
    df_t = df.T  # Transpose directly
    df_t.index.name = 'Sample_ID'
    df_t.columns.name = 'Features'
    return df_t


mrna_df_t = format_expression_matrix(mrna_df_filtered)
proteo_df_t = format_expression_matrix(proteo_df_filtered)
phospho_df_t = format_expression_matrix(phospho_df_filtered)

In [3]:
# PCA function
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import plotly.express as px


def perform_pca_and_plot(df, title, metadata_df):
    df = df.loc[df.index.intersection(metadata_df.index)]
    labels = metadata_df.loc[df.index, 'Label']

    imputer = SimpleImputer(strategy='mean')
    df_pca_imputed = imputer.fit_transform(df)

    scaler = StandardScaler()
    df_pca_scaled = scaler.fit_transform(df_pca_imputed)

    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(df_pca_scaled)
    explained_variance = pca.explained_variance_ratio_ * 100

    pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'], index=df.index)
    pca_df['Label'] = labels.values

    fig = px.scatter(
        pca_df, x='PC1', y='PC2', color='Label', title=f'PCA of {title}',
        labels={'color': 'Sample Label'}, hover_name=pca_df.index
    )
    fig.update_traces(marker=dict(size=10, line=dict(width=2, color='DarkSlateGrey')))
    fig.update_layout(
        autosize=False, width=400, height=300,
        xaxis_title=f'PC 1 ({explained_variance[0]:.2f}% Variance)',
        yaxis_title=f'PC 2 ({explained_variance[1]:.2f}% Variance)',
        title=f'PCA Analysis of {title}'
    )
    fig.show()

# Perform PCA on each dataset
perform_pca_and_plot(mrna_df_t, 'RNA Data', metadata_df)
perform_pca_and_plot(proteo_df_t, 'Protein Data', metadata_df)
perform_pca_and_plot(phospho_df_t, 'Phospho Data', metadata_df)


/Users/dipalikale/Documents/Elucidata/personal_git/multiomics/.venv/lib/python3.11/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['AC095030.1' 'AC113347.1' 'AC122138.1' 'AL023574.1' 'AL356432.2'
 'ATP2B2-IT1' 'LINC00392' 'LINC00613' 'LINC02113' 'LINC02117' 'MTCYBP36'
 'OR4F14P' 'OR6M2P' 'RARRES2P3' 'RPL17P41' 'TUBB8P4']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


/Users/dipalikale/Documents/Elucidata/personal_git/multiomics/.venv/lib/python3.11/site-packages/sklearn/impute/_base.py:635: UserWarning:

Skipping features without any observed values: ['A0A0G2JMI3' 'A6BM72' 'A6NLP5' 'C9JVW0' 'O14598' 'O14788' 'O15131'
 'O15540' 'O43186' 'O75712' 'O75795' 'P01036' 'P01135' 'P01762' 'P03989'
 'P04155' 'P0DP02' 'P15086' 'P30460' 'P30484' 'P30487;P30488' 'P30492'
 'P30990' 'P57738' 'Q02045' 'Q07837' 'Q13021' 'Q15759' 'Q16473' 'Q16553'
 'Q499Z3' 'Q5SSQ6' 'Q5SWH9' 'Q5SWW7' 'Q5T754' 'Q695T7' 'Q6NUN0' 'Q6UWQ7'
 'Q7Z7C7' 'Q86VH4' 'Q8IY42' 'Q8N729' 'Q8NFT8' 'Q8TBE0' 'Q8WVK7' 'Q8WW33'
 'Q8WY98' 'Q96GU1' 'Q96KN3' 'Q96N76' 'Q96NJ1' 'Q96NZ9' 'Q9BSG5' 'Q9BYD5'
 'Q9BYV1' 'Q9H2X3' 'Q9H649' 'Q9H707' 'Q9HBU1' 'Q9NRC9' 'Q9NZ42' 'Q9UBF1'
 'Q9UJJ7' 'Q9UJM8' 'Q9UKR0' 'Q9ULW8' 'Q9UPI3' 'Q9Y258' 'Q9Y5L2']. At least one non-missing value is needed for imputation with strategy='mean'.

